In [1]:
from weight_avg_trees import *
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

device_in_use = 'cuda'

In [2]:
# Load the data
ch = load_breast_cancer()
df = pd.DataFrame(data=ch.data, columns=ch.feature_names)
# Assuming `ch.target` is the target variable
df['Target'] = ch.target

# Splitting the dataset
df_train, df_temp = train_test_split(df, train_size=0.70, random_state=42)
df_val, df_test = train_test_split(df_temp, train_size=0.5, random_state=42)

# Separate the target variable
y_train = df_train['Target'].values
y_val = df_val['Target'].values
y_test = df_test['Target'].values

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler to the training data (excluding the target variable) and transform it
scaled_train_features = scaler.fit_transform(df_train.drop(columns=['Target']))

# Transform the validation and test data (excluding the target variable)
scaled_val_features = scaler.transform(df_val.drop(columns=['Target']))
scaled_test_features = scaler.transform(df_test.drop(columns=['Target']))

# Recombine scaled features with target variable
df_scaled_train = pd.DataFrame(scaled_train_features, columns=df_train.columns[:-1])  # Excluding the target variable column
df_scaled_train['Target'] = y_train

df_scaled_val = pd.DataFrame(scaled_val_features, columns=df_val.columns[:-1])
df_scaled_val['Target'] = y_val

df_scaled_test = pd.DataFrame(scaled_test_features, columns=df_test.columns[:-1])
df_scaled_test['Target'] = y_test

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.features = dataframe.drop('Target', axis=1).values
        self.labels = dataframe['Target'].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float), torch.tensor(self.labels[idx], dtype=torch.long)

train_dataset = CustomDataset(df_scaled_train)
val_dataset = CustomDataset(df_scaled_val)
test_dataset = CustomDataset(df_scaled_test)

train_loader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [3]:
# Define a function to move an entire dataset to the device in advance
def preload_dataset_to_device(loader, batch_size, device):
    preloaded_data = [(inputs.to(device), targets.to(device)) for inputs, targets in loader]
    return DataLoader(preloaded_data, batch_size=batch_size)

# Preload datasets to device (if they fit into your device memory)
train_loader = preload_dataset_to_device(train_loader, len(train_dataset), device_in_use)
val_loader = preload_dataset_to_device(val_loader, len(val_dataset), device_in_use)

In [10]:
input_size = scaled_train_features.shape[1]
output_size = 2

train_loss_traditional = []
test_loss_traditional = []

train_loss_2 = []
test_loss_2 = []

train_loss_4 = []
test_loss_4 = []

train_loss_8 = []
test_loss_8 = []

#105 is the lcm(3,7,15)

from tqdm import tqdm
for i in tqdm(range(100)):

    testloss, trainloss = regular_classification(105, train_loader, val_loader, input_size, output_size, device_in_use, model = 'NN') 

    train_loss_traditional.append(trainloss) 
    test_loss_traditional.append(testloss)

    testloss, trainloss = weight_avg_2_classification(105, train_loader, val_loader, input_size, output_size, device_in_use, model = 'NN') 

    train_loss_2.append(trainloss) 
    test_loss_2.append(testloss)

    testloss, trainloss = weight_avg_4_classification(105, train_loader, val_loader, input_size, output_size, device_in_use, model = 'NN') 

    train_loss_4.append(trainloss) 
    test_loss_4.append(testloss)

    testloss, trainloss = weight_avg_8_classification(105, train_loader, val_loader, input_size, output_size, device_in_use, model = 'NN') 

    train_loss_8.append(trainloss) 
    test_loss_8.append(testloss)


print("Traditional:",np.mean(test_loss_traditional),np.std(test_loss_traditional))
print("Weight Avg 2 Base Models:",np.mean(test_loss_2),np.std(test_loss_2))
print("Weight Avg 4 Base Models:",np.mean(test_loss_4),np.std(test_loss_4))
print("Weight Avg 8 Base Models:",np.mean(test_loss_8),np.std(test_loss_8))

100%|██████████| 100/100 [01:47<00:00,  1.08s/it]

Traditional: 0.1710547363758087 0.0034637396087530033
Weight Avg 2 Base Models: 0.2943999773263931 0.005980698999644444
Weight Avg 4 Base Models: 0.46070484310388565 0.008638467151526335
Weight Avg 8 Base Models: 0.5933123034238815 0.007556758551685531


In [9]:
input_size = scaled_train_features.shape[1]
output_size = 2

train_loss_traditional = []
test_loss_traditional = []

train_loss_2 = []
test_loss_2 = []

train_loss_4 = []
test_loss_4 = []

train_loss_8 = []
test_loss_8 = []

#105 is the lcm(3,7,15)

from tqdm import tqdm
for i in tqdm(range(100)):

    testloss, trainloss = regular_classification(105, train_loader, val_loader, input_size, output_size, device_in_use, model = 'Linear') 

    train_loss_traditional.append(trainloss) 
    test_loss_traditional.append(testloss)

    testloss, trainloss = weight_avg_2_classification(105, train_loader, val_loader, input_size, output_size, device_in_use, model = 'Linear') 

    train_loss_2.append(trainloss) 
    test_loss_2.append(testloss)

    testloss, trainloss = weight_avg_4_classification(105, train_loader, val_loader, input_size, output_size, device_in_use, model = 'Linear') 

    train_loss_4.append(trainloss) 
    test_loss_4.append(testloss)

    testloss, trainloss = weight_avg_8_classification(105, train_loader, val_loader, input_size, output_size, device_in_use, model = 'Linear') 

    train_loss_8.append(trainloss) 
    test_loss_8.append(testloss)


print("Traditional:",np.mean(test_loss_traditional),np.std(test_loss_traditional))
print("Weight Avg 2 Base Models:",np.mean(test_loss_2),np.std(test_loss_2))
print("Weight Avg 4 Base Models:",np.mean(test_loss_4),np.std(test_loss_4))
print("Weight Avg 8 Base Models:",np.mean(test_loss_8),np.std(test_loss_8))

100%|██████████| 100/100 [01:17<00:00,  1.30it/s]

Traditional: 0.7533269420266151 0.2244393126898861
Weight Avg 2 Base Models: 0.7639818024635315 0.1907219393882405
Weight Avg 4 Base Models: 0.702273827791214 0.12840718709104437
Weight Avg 8 Base Models: 0.6977672284841537 0.07924951804093364


In [7]:

# Initialize the model
model = LinearModel(input_size, output_size).to(device_in_use)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent

for epoch in range(105):
    model.train()  

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device_in_use), targets.to(device_in_use)
        optimizer.zero_grad()  
        outputs = model(inputs) 
        loss = criterion(outputs.squeeze(0), targets.squeeze(0))  
        loss.backward() 
        optimizer.step()  

    train_loss = loss


    model.eval()  

    with torch.no_grad(): 
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device_in_use), targets.to(device_in_use)
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(0), targets.squeeze(0))

    test_loss = loss
    
    print("Train", train_loss.item(), "Test", test_loss.item())

Train 0.7016337513923645 Test 0.6464226245880127
Train 0.6630672812461853 Test 0.6075065732002258
Train 0.6288616061210632 Test 0.573322057723999
Train 0.5985026359558105 Test 0.5432292819023132
Train 0.5714974403381348 Test 0.516645073890686
Train 0.5473955273628235 Test 0.49305787682533264
Train 0.5257981419563293 Test 0.47202885150909424
Train 0.5063608288764954 Test 0.4531875550746918
Train 0.4887898564338684 Test 0.43622323870658875
Train 0.47283613681793213 Test 0.4208759069442749
Train 0.45828938484191895 Test 0.40692803263664246
Train 0.4449717700481415 Test 0.3941969573497772
Train 0.4327324330806732 Test 0.3825294077396393
Train 0.42144352197647095 Test 0.37179553508758545
Train 0.4109960198402405 Test 0.3618853986263275
Train 0.40129661560058594 Test 0.3527051508426666
Train 0.3922649025917053 Test 0.34417474269866943
Train 0.38383159041404724 Test 0.33622488379478455
Train 0.3759368062019348 Test 0.3287960886955261
Train 0.3685280978679657 Test 0.32183653116226196
Train 0.3